In [1]:
print(1)
import os
import sys
import shutil
# Get directory name
mydir= 'tf_logs'

## Try to remove tree; if failed show an error using try...except on screen
try:
    shutil.rmtree(mydir)
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))

1
Error: tf_logs\events.out.tfevents.1548261128.DESKTOP-1QLQT1T - Access is denied.


In [1]:
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline
from multiprocessing import Process,Queue,Value
from utils.Preprocessor import Frame_Processor
from utils.generator import generate
import keras

Using TensorFlow backend.


In [2]:
crop_coords={"top_left_x":4,"top_left_y":8,"bottom_right_x":-12,"bottom_right_y":-12}
FP=Frame_Processor(crop=True,convert_to_grayscale=True,normalize=True,coords=crop_coords,resize=True,resize_y=110,resize_x=84)
replay_buffer=Queue(maxsize=1000)
saved_new=Value('i',0)

In [3]:
config={'env_name':'SpaceInvaders-v0','nstack':4,'gain':5,'decay':1e-6,
        'max_episode_steps':10000,
        'replay_buffer':replay_buffer,'Frame_Processor':FP,'gamma':0.9,'saved_new':saved_new}

In [4]:
P=Process(target=generate,args=(config,))
P.start()

In [5]:
replay_buffer.get()[0].shape

(1, 110, 84, 4)

In [6]:
from keras.optimizers import Adam
from utils.PolicyNetwork import DQN
input_shape=(110,84,4)#env.obs_shape
num_outputs=6
optimizer=Adam(lr=0.0001)
design_config_file=r"utils\qnetwork.csv"
model=DQN(input_shape=input_shape,num_outputs=num_outputs,
        optimizer=optimizer,arch_config_file=design_config_file)
model.construct_model(training=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 110, 84, 4)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 26, 20, 32)   8224        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 26, 20, 32)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 12, 9, 64)    32832       activation_1[0][0]               
__________________________________________________________________________________________________
activation

C:\Users\MuhammadUsman\PycharmProjects\Deep-Q-Learning\utils\PolicyNetwork.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("la...)`
  self.Model = keras_model(inputs=[state_inputs,action_one_hot_inputs], output=pred_q_a)


In [7]:
def sample(replay_buffer,steps):
    Data=[]
    y=[]
    a=[]
    for i in range(steps):
        point=replay_buffer.get()
        Data.append(point[0][0])
        y.append(point[1])
        a.append(point[2])
    return np.array(Data),np.array(y),np.array(a)

In [8]:
for i in range(100):
    print (i)
    X,y,a=sample(replay_buffer,1000)
    model.Model.fit([X,keras.utils.to_categorical(a,num_classes=6)],y,epochs=10,batch_size=5,
                    verbose=0)
    model.Model.save_weights('weights.h5')
    with saved_new.get_lock():
        saved_new.value = 1

0


1


2


3


4


...

6


7


8


9


10


In [10]:
with saved_new.get_lock():
    saved_new.value=1

In [11]:
replay_buffer.get()

(array([[[[0.        , 0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        , 0.        ]],
 
         [[0.00151918, 0.00151918, 0.00151918, 0.00151918],
          [0.00151918, 0.00151918, 0.00151918, 0.00151918],
          [0.00151918, 0.00151918, 0.00151918, 0.00151918],
          ...,
          [0.00147554, 0.00147554, 0.00147554, 0.00147554],
          [0.        , 0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        , 0.        ]],
 
         [[0.00136726, 0.00136726, 0.00136726, 0.00136726],
          [0.00136726, 0.00136726, 0.00136726, 0.00136726],
          [0.00029298, 0.00029298, 0.00029298, 0.00029298],
          ...,
          [0.00132799, 0.00132799, 0.00132799, 0.